In [1]:
import pandas as pd
import mne
import numpy as np
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from openpyxl import load_workbook
import pickle

In [2]:
pd.set_option('display.max_columns', 100)  # or 1000
pd.set_option('display.max_rows', 100)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [3]:
# path for EEG raw data
datapaths = ['/data/sleep_germandata_reactivation-1/']
files=[]

while datapaths:
    # paths.pop() gets top of directory stack to process
    # os.scandir is easier and more efficient than os.listdir
    with os.scandir(datapaths.pop()) as entries:
        for entry in entries:  # loop through the folder
            #print(entry.name)  # print text to keep track the process
            if entry.name.endswith('.fif') and '-1' not in entry.name:
                files.append(entry.path)
            elif entry.is_dir():  #if it is a subfolder
                #print(entry.path, 'is dir')
                # Add to paths stack to get to it eventually
                datapaths.append(entry.path)

In [4]:
len(files)

46

In [5]:
cond='sleep'
fname=[] # data for sleep
fname_D=[] 
fname_M=[] # control
subjs=[]
for file in files:
    if cond in file:
        fname.append(file)
    # separate into D and M Night
        subj=file.split('/')[-1].split('_')[1][:2]
        night=float(file.split('/')[-1].split('_')[1][-1])
        if float(subj) in np.arange(5,31) and night==1:
            fname_D.append(file)
        elif float(subj) in np.arange(5,31) and night==2:
            fname_M.append(file)
        elif float(subj) in np.arange(39,52) and night==2:
            fname_D.append(file)
        elif float(subj) in np.arange(39,52) and night==1:
            fname_M.append(file)
        subjs.append(subj)
            
subjs=list(set(subjs))

In [6]:
print(len(fname_D))
print(len(fname_M))

23
23


In [7]:
# path for Event excel files
datapaths = ['./Events']
events_files=[]

while datapaths:
    # paths.pop() gets top of directory stack to process
    # os.scandir is easier and more efficient than os.listdir
    with os.scandir(datapaths.pop()) as entries:
        for entry in entries:  # loop through the folder
            #print(entry.name)  # print text to keep track the process
            if entry.name.endswith('.xlsx'):
                events_files.append(entry.path)
            elif entry.is_dir():  #if it is a subfolder
                #print(entry.path, 'is dir')
                # Add to paths stack to get to it eventually
                datapaths.append(entry.path)

In [8]:
len(events_files)

46

In [9]:
#fname_D

In [10]:
#fname_M

## Loop to load EEG, Event files and save EEG arrays

In [11]:
from collections import defaultdict 
from tensorflow.keras.utils import to_categorical
#from sklearn.model_selection import StratifiedKFold
#import sys
#sys.path.append('./arl-eegmodels')
#from EEGModels import EEGNet
#from sklearn.metrics import roc_auc_score
#from sklearn.metrics import accuracy_score
#from sklearn.metrics import f1_score
#from sklearn.model_selection import train_test_split
#from tensorflow.keras.callbacks import EarlyStopping

2024-06-28 12:45:31.783797: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
folder='./eeg_sub_128_SWS/'

## D night

In [15]:
#accD=defaultdict(dict)
#f1D=defaultdict(dict)
for s in subjs:
    print(' ')
    print(f"Subj {s}:")
    # D night
    for fD in fname_D:
        if 'RC_'+s in fD:
            file_D=fD
            print(file_D)
            # load EEG        
            raw_data= mne.io.read_raw(file_D,preload=True,verbose=0)
            raw_data=raw_data.resample(128,npad='auto',n_jobs=-1,verbose=0) # downsample
            raw_data=raw_data.filter(l_freq=0.5, h_freq=4.5,n_jobs=-1,verbose=0) # band-pass filter 
            eeg_data=raw_data.pick(picks=['eeg'])
            eeg_df=eeg_data.to_data_frame(time_format='ms').iloc[:,:-1] # remove channel 129
            # load Event file for file_D
            events_fn=file_D.split('/')[-1].split('_')[1]
            for ff in events_files:
                if events_fn in ff:
                    print(ff)
                    events_info=pd.read_excel(ff,engine='openpyxl',usecols='A:M')
            events_df=events_info[['sample','stimulation','Rejected']]
            events_df=events_df[events_df['Rejected']==0]
   
            del raw_data
            del eeg_data
    
            # match timing for EEG and events
            events_df['time']=0
            for e in events_df.index:
                sample=events_df['sample'][e]
                abs_diff=abs(eeg_df['time']-sample)
                closest_row_index = abs_diff.idxmin()
                events_df['time'][e]=eeg_df['time'][closest_row_index]
    
            # separate events into odor and vehicle
            odor_df=events_df[events_df['stimulation']=='ODOR'].reset_index(drop=True)
            veh_df=events_df[events_df['stimulation']=='VEHICLE'].reset_index(drop=True)
            #print(len(odor_df))
            #print(len(veh_df))
    
            # separate EEG epochs into odor and vehicle
            eeg_odor_stim=[]
            eeg_veh_stim=[]
    
            trial_no=min([len(odor_df),len(veh_df)])
            for tt in range(trial_no):
                odor_start=odor_df['time'][tt] #odor start
                veh_start=veh_df['time'][tt] #vehicle start
   
                # EEG epoch for odor or vehicle (theta epoch:-1 to 5s; slow/fast spindles:9-12.5/12.5-16 Hz)                eeg_odor_df=eeg_df[(eeg_df['time']>=odor_start) & (eeg_df['time']<=(odor_start+7000))].iloc[:,1:].transpose()
                eeg_veh_df=eeg_df[(eeg_df['time']>=veh_start) & (eeg_df['time']<=(veh_start+7000))].iloc[:,1:].transpose()
        
            # convert EEG dataframes into 3D arrays
                eeg_odor=eeg_odor_df.to_numpy()[np.newaxis,:, :]
                eeg_veh=eeg_veh_df.to_numpy()[np.newaxis,:, :]
        
            # concatenate EEG epochs/arrays
                if tt==0:
                    eeg_odor_stim=eeg_odor
                    eeg_veh_stim=eeg_veh
                elif tt>0:
                    eeg_odor_stim=np.concatenate((eeg_odor_stim,eeg_odor), axis=0)
                    eeg_veh_stim=np.concatenate((eeg_veh_stim,eeg_veh), axis=0)
            XD=np.concatenate((eeg_veh_stim,eeg_odor_stim), axis=0)
            print(XD.shape)
            #XD_index=np.arange(0,XD.shape[0])
            
            #Ground truth
            y_odor=np.ones(trial_no) # 1 is odor
            y_veh=np.zeros(trial_no) # 0 is vehicle
            y=np.concatenate((y_veh,y_odor))
            y_encodedD=to_categorical(y-1,num_classes=2)
            #print(y_encodedD.shape)
            
            # save EEG array for each subject
            with open(folder+'XD_'+s+'.npy', 'wb') as f:
                np.save(f, XD)
    
            with open(folder+'y_encodedD_'+s+'.npy', 'wb') as f:
                np.save(f, y_encodedD)

 
Subj 45:
/data/sleep_germandata_reactivation-1/RC_452_sleep_20170622_121802.fif


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 102 out of 137 | elapsed:   18.0s remaining:    6.2s
[Parallel(n_jobs=-1)]: Done 130 out of 137 | elapsed:   20.8s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done 137 out of 137 | elapsed:   21.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 102 out of 137 | elapsed:    4.3s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done 130 out of 137 | elapsed:    5.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 137 out of 137 | elapsed:    5.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  92 out of 129 | elapsed:    1.3s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done 118 out 

./Events/RC_452.xlsx
(40, 128, 897)
 
Subj 46:
/data/sleep_germandata_reactivation-1/RC_462_sleep_20170630_120717.fif


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 102 out of 137 | elapsed:   10.0s remaining:    3.4s
[Parallel(n_jobs=-1)]: Done 130 out of 137 | elapsed:   12.6s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done 137 out of 137 | elapsed:   13.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 102 out of 137 | elapsed:    0.4s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 130 out of 137 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 137 out of 137 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  92 out of 129 | elapsed:    0.7s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 118 out 

./Events/RC_462.xlsx
(54, 128, 897)
 
Subj 48:
/data/sleep_germandata_reactivation-1/RC_482_sleep_20170725_115657.fif


...

./Events/RC_251.xlsx
(78, 128, 897)
 
Subj 14:
/data/sleep_germandata_reactivation-1/RC_141_sleep_20160621_123516.fif


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 102 out of 137 | elapsed:    9.5s remaining:    3.3s
[Parallel(n_jobs=-1)]: Done 130 out of 137 | elapsed:   12.1s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done 137 out of 137 | elapsed:   12.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 102 out of 137 | elapsed:    3.5s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done 130 out of 137 | elapsed:    4.4s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 137 out of 137 | elapsed:    4.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  92 out of 129 | elapsed:    0.8s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 118 out 

./Events/RC_141.xlsx
(44, 128, 897)
 
Subj 39:
/data/sleep_germandata_reactivation-1/RC_392_sleep_20170517_120847.fif


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 102 out of 137 | elapsed:    9.6s remaining:    3.3s
[Parallel(n_jobs=-1)]: Done 130 out of 137 | elapsed:   12.4s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done 137 out of 137 | elapsed:   13.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 102 out of 137 | elapsed:    0.3s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 130 out of 137 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 137 out of 137 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  92 out of 129 | elapsed:    0.6s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 118 out 

./Events/RC_392.xlsx
(66, 128, 897)


## M night

In [16]:
#accM=defaultdict(dict)
#f1M=defaultdict(dict)
for s in subjs:
    # M night
    print(' ')
    print(f"Subj {s}:")
    for fM in fname_M:
        if 'RC_'+s in fM:
            file_M=fM
            print(file_M)
            # load EEG        
            raw_data= mne.io.read_raw(file_M,preload=True,verbose=0)
            raw_data=raw_data.resample(128,npad='auto',n_jobs=-1,verbose=0) # downsample
            raw_data=raw_data.filter(l_freq=0.5, h_freq=4.5,n_jobs=-1,verbose=0) # band-pass filter for sleep spindles
            eeg_data=raw_data.pick(picks=['eeg'])
            eeg_df=eeg_data.to_data_frame(time_format='ms').iloc[:,:-1] # remove channel 129
            # load Event file for file_M
            events_fn=file_M.split('/')[-1].split('_')[1]
            for ff in events_files:
                if events_fn in ff:
                    print(ff)
                    events_info=pd.read_excel(ff,engine='openpyxl',usecols='A:M')
            events_df=events_info[['sample','stimulation','Rejected']]
            events_df=events_df[events_df['Rejected']==0]
   
            del raw_data
            del eeg_data
    
            # match timing for EEG and events
            events_df['time']=0
            for e in events_df.index:
                sample=events_df['sample'][e]
                abs_diff=abs(eeg_df['time']-sample)
                closest_row_index = abs_diff.idxmin()
                events_df['time'][e]=eeg_df['time'][closest_row_index]
    
            # separate events into odor and vehicle
            odor_df=events_df[events_df['stimulation']=='ODOR'].reset_index(drop=True)
            veh_df=events_df[events_df['stimulation']=='VEHICLE'].reset_index(drop=True)
            #print(len(odor_df))
            #print(len(veh_df))
    
            # separate EEG epochs into odor and vehicle
            eeg_odor_stim=[]
            eeg_veh_stim=[]
    
            trial_no=min([len(odor_df),len(veh_df)])
            for tt in range(trial_no):
                odor_start=odor_df['time'][tt] #odor start
                veh_start=veh_df['time'][tt] #vehicle start
   
                # EEG epoch for odor or vehicle
                eeg_odor_df=eeg_df[(eeg_df['time']>=odor_start) & (eeg_df['time']<=(odor_start+7000))].iloc[:,1:].transpose()
                eeg_veh_df=eeg_df[(eeg_df['time']>=veh_start) & (eeg_df['time']<=(veh_start+7000))].iloc[:,1:].transpose()
        
            # convert EEG dataframes into 3D arrays
                eeg_odor=eeg_odor_df.to_numpy()[np.newaxis,:, :]
                eeg_veh=eeg_veh_df.to_numpy()[np.newaxis,:, :]
        
            # concatenate EEG epochs/arrays
                if tt==0:
                    eeg_odor_stim=eeg_odor
                    eeg_veh_stim=eeg_veh
                elif tt>0:
                    eeg_odor_stim=np.concatenate((eeg_odor_stim,eeg_odor), axis=0)
                    eeg_veh_stim=np.concatenate((eeg_veh_stim,eeg_veh), axis=0)
            XM=np.concatenate((eeg_veh_stim,eeg_odor_stim), axis=0)
            print(XM.shape)
            #XM_index=np.arange(0,XM.shape[0])
            
            #Ground truth
            y_odor=np.ones(trial_no) # 1 is odor
            y_veh=np.zeros(trial_no) # 0 is vehicle
            y=np.concatenate((y_veh,y_odor))
            y_encodedM=to_categorical(y-1,num_classes=2)
            #print(y_encodedM.shape)
            
            # save EEG array for each subject
            with open(folder+'XM_'+s+'.npy', 'wb') as f:
                np.save(f, XM)
    
            with open(folder+'y_encodedM_'+s+'.npy', 'wb') as f:
                np.save(f, y_encodedM)

 
Subj 45:
/data/sleep_germandata_reactivation-1/RC_451_sleep_20170608_122254.fif


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 102 out of 137 | elapsed:    9.6s remaining:    3.3s
[Parallel(n_jobs=-1)]: Done 130 out of 137 | elapsed:   12.3s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done 137 out of 137 | elapsed:   12.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 102 out of 137 | elapsed:    3.8s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done 130 out of 137 | elapsed:    4.7s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 137 out of 137 | elapsed:    4.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  92 out of 129 | elapsed:    0.9s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done 118 out 

./Events/RC_451.xlsx
(52, 128, 897)
 
Subj 46:
/data/sleep_germandata_reactivation-1/RC_461_sleep_20170615_121704.fif


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 102 out of 137 | elapsed:    9.2s remaining:    3.1s
[Parallel(n_jobs=-1)]: Done 130 out of 137 | elapsed:   11.6s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done 137 out of 137 | elapsed:   12.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  92 out of 129 | elapsed:    0.5s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 118 out of 129 | elapsed:    0.7s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 129 out of 129 | elapsed:    0.8s finished


./Events/RC_461.xlsx
(50, 128, 897)
 
Subj 48:
/data/sleep_germandata_reactivation-1/RC_481_sleep_20170712_121519.fif


...

./Events/RC_252.xlsx
(60, 128, 897)
 
Subj 14:
/data/sleep_germandata_reactivation-1/RC_142_sleep_20160705_112314.fif


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 102 out of 137 | elapsed:   10.6s remaining:    3.6s
[Parallel(n_jobs=-1)]: Done 130 out of 137 | elapsed:   13.3s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done 137 out of 137 | elapsed:   13.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 102 out of 137 | elapsed:    4.3s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done 130 out of 137 | elapsed:    5.5s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 137 out of 137 | elapsed:    5.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  92 out of 129 | elapsed:    0.9s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 118 out 

./Events/RC_142.xlsx
(42, 128, 897)
 
Subj 39:
/data/sleep_germandata_reactivation-1/RC_391_sleep_20170501_120806.fif


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 102 out of 137 | elapsed:    9.7s remaining:    3.3s
[Parallel(n_jobs=-1)]: Done 130 out of 137 | elapsed:   12.4s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done 137 out of 137 | elapsed:   13.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 102 out of 137 | elapsed:    7.0s remaining:    2.4s
[Parallel(n_jobs=-1)]: Done 130 out of 137 | elapsed:    8.9s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done 137 out of 137 | elapsed:    9.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  92 out of 129 | elapsed:    0.9s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done 118 out 

./Events/RC_391.xlsx
(54, 128, 897)
